## Day 18 - Lava pits

**Part 1: Just draw the lava pit based on instructions and say how much lava fit in it**

In [1]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['R 6 (#70c710)',
 'D 5 (#0dc571)',
 'L 2 (#5713f0)',
 'D 2 (#d2c081)',
 'R 2 (#59c680)',
 'D 2 (#411b91)',
 'L 5 (#8ceee2)',
 'U 2 (#caa173)',
 'L 1 (#1b58a2)',
 'U 2 (#caa171)',
 'R 2 (#7807d2)',
 'U 3 (#a77fa3)',
 'L 2 (#015232)',
 'U 2 (#7a21e3)']

In [2]:
import sys

sys.setrecursionlimit(int(1e9))

def flood_fill_recursive(y, x, grid: list[list[int]]):
    border_x = len(grid[0]) - 1
    border_y = len(grid) - 1

    saw_along_the_journey = set()

    some_dumb_list = []
    def lets_try_to_escape(y, x, the_dumb_list: list):
        """
        Using flood_fill idea to define if we can escape the grid?
        """
        
        if not (0 < y < border_y and 0 < x < border_x):
            # hit an edge!
            # tried to use a global bool here but was messing up
            if not grid[y][x]:
                the_dumb_list.append(True)
                return
        else:
            saw_along_the_journey.add((y, x))

        if (next_y:= y+1, next_x:= x) not in saw_along_the_journey:
            if not grid[next_y][next_x]:
                lets_try_to_escape(next_y, next_x, the_dumb_list)
        if (next_y:= y-1, next_x:= x) not in saw_along_the_journey:
            if not grid[next_y][next_x]:
                lets_try_to_escape(next_y, next_x, the_dumb_list)
        if (next_y:= y, next_x:= x+1) not in saw_along_the_journey:
            if not grid[next_y][next_x]:
                lets_try_to_escape(next_y, next_x, the_dumb_list)
        if (next_y:= y, next_x:= x-1) not in saw_along_the_journey:
            if not grid[next_y][next_x]:
                lets_try_to_escape(next_y, next_x, the_dumb_list)
    
    lets_try_to_escape(y, x, some_dumb_list)

    if True in some_dumb_list:
        return True, saw_along_the_journey
    else:
        return False, saw_along_the_journey

In [3]:
def build_grid_from_trenches(trenches: list[tuple[int, int]]) -> list[list[int]]:
    min_x = min(x for _,x in trenches)
    min_y = min(y for y,_ in trenches)
    max_x = max(x for _,x in trenches)
    max_y = max(y for y,_ in trenches)

    # Note: actual max index not like a python range upper bound
    grid = [[0 for _ in range(min_x, max_x+1)] for _ in range(min_y, max_y+1)]
    for ty, tx in trenches:
        # we should shift our grid so it starts at (0,0) else everything else I've been doing bloody breaks obviously!
        grid[ty+abs(min_y)][tx+abs(min_x)] = 1

    return grid

In [4]:
def part1(lines: list[str]) -> int:
    starting_position = (0,0)
    trenches = [starting_position]
    current_position = starting_position
    for line in lines:
        direction, dist, _ = line.split()  # Colour not used at least in part1
        dist = int(dist)
        current_y, current_x = current_position
        for _ in range(1, dist+1):
            if direction == "R":
                current_x += 1
            elif direction == "L":
                current_x -= 1
            elif direction == "U":
                current_y -= 1
            elif direction == "D":
                current_y += 1

            current_position = (current_y, current_x)
            trenches.append(current_position)

    grid = build_grid_from_trenches(trenches)

    non_borders_to_check = [
        (y,x)
        for y in range(len(grid))
        for x in range(len(grid[0]))
        if grid[y][x] != 1
    ]

    assert len(grid[0])*len(grid) == len(set(trenches)) + len(non_borders_to_check)

    while non_borders_to_check:
        y, x = non_borders_to_check.pop()
        escaped, saw_along_the_journey = flood_fill_recursive(y, x, grid)
        non_borders_to_check = [item for item in non_borders_to_check if item not in saw_along_the_journey]
        if not escaped:
            for saw_y, saw_x in saw_along_the_journey:
                grid[saw_y][saw_x] = 1

    flattened_grid = []
    for row in grid:
        flattened_grid.extend(row)

    return flattened_grid.count(1)

assert part1(example_lines) == 62
part1(input_lines)

45159

omg I did it

**Part 2: ...**